Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give read access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


In [ ]:
## DONT CHANGE THIS CELL 
# this is currently the same as dev.data.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt

--2021-10-11 18:13:10--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt [following]
--2021-10-11 18:13:11--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63639 (62K) [text/plain]
Saving to: ‘test.data.txt’

test.data.txt       100%[===================>]  62.15K  87.9KB/s    in 0.7s    

2021-10-11 18:13:13 (87.9 KB/s) - ‘test.data.txt’ saved [63639/63639]



In [ ]:
## Replace with the right link that contains the zip file uploaded from the training
!gdown https://drive.google.com/uc?id=1-4LLmFgkzKL63dSVmbCFPUR5MOupTXlw

# https://drive.google.com/file/d/1-4LLmFgkzKL63dSVmbCFPUR5MOupTXlw/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1-4LLmFgkzKL63dSVmbCFPUR5MOupTXlw
To: /content/2018MT10770_A_model.zip
100% 4.24G/4.24G [01:16<00:00, 55.2MB/s]


In [ ]:
!unzip 2018MT10770_A_model.zip

Archive:  2018MT10770_A_model.zip
   creating: models/
   creating: models/.ipynb_checkpoints/
  inflating: models/model-1633966417.4227178.pt  


In [ ]:
## Import relevant packages

import os
import logging
import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as O
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

import torchtext
from torchtext import datasets, vocab
from torchtext.legacy import data
from torchtext.vocab import GloVe

from argparse import ArgumentParser
from pdb import set_trace
from typing import Dict, Tuple, List, Union, Optional
from tqdm import tqdm
import time, datetime
import string

import random


In [ ]:
# set seeds
random.seed(772)
torch.manual_seed(772)
torch.cuda.manual_seed(772)

In [ ]:
## Various utility functions

"""checking arguments"""
def check_args(args):
	# --tmp_dir
	check_folder(args.tmp_dir)

	# --data_path
	assert os.path.isfile(args.data_path), "data_path not found"

	# --saved_model
	assert os.path.isfile(args.saved_model), "saved_model not found"

	return args

def makedirs(name):
	"""helper function for python 2 and 3 to call os.makedirs()
		avoiding an error if the directory to be created already exists"""

	import os, errno

	try:
		os.makedirs(name)
	except OSError as ex:
		if ex.errno == errno.EEXIST and os.path.isdir(name):
			# ignore existing directory
			pass
		else:
			# a different error happened
			raise

def check_folder(log_dir):
	if not os.path.exists(log_dir):
		os.makedirs(log_dir)
	return log_dir


In [ ]:
def parse_args():
  parser = ArgumentParser(description='Word Meaning Comparison')

  parser.add_argument('--data_path', '-d', type=str, default='test.data.txt')
  parser.add_argument('--saved_model', '-m', type=str, default='/content/models/model-1633966417.4227178.pt')
  parser.add_argument('--out_path', '-o', type=str, default='output.txt')

  parser.add_argument('--tmp_dir', type=str, default='tmp')

  parser.add_argument('--gpu', type=int, default=0)

  return check_args(parser.parse_known_args()[0])

args = parse_args()

In [ ]:
device = torch.device('cuda' if args.gpu and torch.cuda.is_available() else 'cpu')

In [ ]:
class BiLSTM(nn.Module):
  def __init__(
      self,
      embedding: torch.Tensor,
      rnn_hidden_dim: int,
      rnn_layers: int,
      dropout: float = 0,
      freeze_emb: bool = True
  ) -> None:

    super().__init__()

    assert embedding is not None
    self.embedding = nn.Embedding.from_pretrained(embedding.vectors, freeze=freeze_emb)
    self.emb_size = embedding.vectors.size()[-1]

    self.rnn_hidden_dim = rnn_hidden_dim
    self.encoder = nn.LSTM(
            self.emb_size, rnn_hidden_dim,
            bidirectional = True,
            num_layers = rnn_layers,
            dropout = (0 if rnn_layers == 1 else dropout),
            batch_first = True
        )
    self.dropout = nn.Dropout(dropout)
    self.sigmoid = nn.Sigmoid()

    # logger.info("New model instantiated with arguments: {}".format(args))
    
  def forward(self, idx1, idx2, s1, s2):
    t = torch.arange(s1.shape[0])

    s1 = self.embedding(s1)
    s1 = self.dropout(s1)
    s1, _ = self.encoder(s1)
    s1 = s1[ :, :, :self.rnn_hidden_dim ] +  s1[ :, :, self.rnn_hidden_dim: ]
    s1 = s1[t, idx1, :].squeeze(1)

    s2 = self.embedding(s2)
    s2 = self.dropout(s2)
    s2, _ = self.encoder(s2)
    s2 = s2[ :, :, :self.rnn_hidden_dim ] +  s2[ :, :, self.rnn_hidden_dim: ]
    s2 = s2[t, idx2, :].squeeze(1)

    out = torch.sum(s1 * s2, dim=-1)/torch.sqrt(torch.sum(s1**2, dim=-1) * torch.sum(s2**2, dim=-1))
    out = 0.5 * (1 + out)
    ##out = torch.sum(s1 * s2, dim=-1)#.unsqueeze(1)   # dot product
    ## out = self.sigmoid(out) # #todo

    return out


In [ ]:
label_encode = {
    'F': 0,
    'T': 1
}

label_decode = {
    0: 'F',
    1: 'T'
}

In [ ]:
## YOUR testing code

def str_split(x):
  return x.split()


save_model = torch.load(args.saved_model, map_location=device)


model = save_model['model']
model.load_state_dict(save_model['model_state'])
model.eval()


BiLSTM(
  (embedding): Embedding(1917496, 300)
  (encoder): LSTM(300, 150, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (sigmoid): Sigmoid()
)

In [ ]:
save_model['args']

Namespace(batch_size=32, data_path='data/train/train.data.txt', data_split=[1.0], dropout=0.5, embed_dim=300, embed_name='42B', epochs=25, gold_path='data/train/train.gold.txt', gpu=1, lr=0.001, lr_decay=0.0005, model='bilstm', models_dir='models', out_dir='out', rnn_hidden_dim=150, rnn_layers=1, tmp_dir='tmp')

In [ ]:
data_path =  os.path.join(args.tmp_dir, "data.tsv")

def process_data():
  combined = pd.read_csv("test.data.txt", sep='\t', header=None)
  combined['idx_l'] = combined.apply(lambda x: x[2].split('-')[0], axis=1).astype(int)
  combined['idx_r'] = combined.apply(lambda x: x[2].split('-')[-1], axis=1).astype(int)
  combined.columns = ['word', 'pos', 'idx', 's1', 's2', 'idx_s1', 'idx_s2']
  combined = combined[['word', 'pos', 'idx_s1', 'idx_s2', 's1', 's2']]
  print(data_path)
  combined.to_csv(data_path, sep='\t', index=False, header=False)
  return combined

process_data()

tmp/data.tsv


,word,pos,idx_s1,idx_s2,s1,s2
0,board,N,2,2,Room and board .,He nailed boards across the windows .
1,circulate,V,0,4,Circulate a rumor .,This letter is being circulated among the facu...
2,hook,V,0,1,Hook a fish .,"He hooked a snake accidentally , and was so sc..."
3,recreation,N,1,9,For recreation he wrote poetry and solved cros...,Drug abuse is often regarded as a form of recr...
4,domesticity,N,4,6,Making a hobby of domesticity .,A royal family living in unpretentious domesti...
...,...,...,...,...,...,...
633,base,N,9,0,He told the painter that he wanted a yellow ba...,Bases include oxides and hydroxides of metals ...
634,power,N,5,6,He needed a lot of power to hit the ball out o...,The mysterious presence of an evil power .
635,portmanteau,N,8,5,` brunch ' is a well - known portmanteau .,` motel ' is a portmanteau word made by combin...
636,promise,V,1,1,I promise you my best effort .,I promised somebody my time .


In [ ]:
def str_split(x):
  return x.split()

# WORD = data.Field(lower=True)
# POS = data.Field(lower=True)
###INDEX = data.Field(sequential=False, use_vocab=False, batch_first=True)
###TEXT = data.Field(sequential=True, tokenize=str_split, lower=True, batch_first=True)
###LABEL = data.LabelField(dtype=torch.int, is_target=True, batch_first=True, use_vocab=False)

INDEX, TEXT, LABEL = save_model['fields']

dataset  = data.TabularDataset(path=data_path, 
                               format='tsv', 
                               fields=[
                                       (None, None), 
                                       (None, None), 
                                       ('idx1', INDEX), 
                                       ('idx2', INDEX), 
                                       ('s1', TEXT), 
                                       ('s2', TEXT), 
                                       #('label', LABEL),
                                       ], skip_header=False)


dt_test = dataset  

print([i for i in dt_test.s1])
# LABEL.build_vocab(dt_test)


it_test = data.BucketIterator(dt_test, batch_size=len(dt_test), device=device, repeat=False, sort=False, shuffle=False)

'''
for b in it_test:
    # print (b.s1, b.s2, b.idx1, b.idx2, b.label)
    #print (b.s1)
    print (b.s1.size())
    print (b.idx1)
    print (b.idx2)
'''
print(len(it_test))


[['room', 'and', 'board', '.'], ['circulate', 'a', 'rumor', '.'], ['hook', 'a', 'fish', '.'], ['for', 'recreation', 'he', 'wrote', 'poetry', 'and', 'solved', 'crossword', 'puzzles', '.'], ['making', 'a', 'hobby', 'of', 'domesticity', '.'], ['the', 'child', "'s", 'acquisition', 'of', 'language', '.'], ['there', 'was', 'no', 'meeting', 'of', 'minds', '.'], ['they', 'swam', 'in', 'the', 'nude', '.'], ['he', 'left', 'an', 'indelible', 'mark', 'on', 'the', 'american', 'theater', '.'], ['conditioning', 'is', 'a', 'form', 'of', 'learning', 'by', 'association', '.'], ['the', 'alkaline', 'inclination', 'of', 'the', 'local', 'waters', '.'], ['glaze', 'the', 'bread', 'with', 'eggwhite', '.'], ['an', 'amendment', 'to', 'piggyback', 'the', 'current', 'law', '.'], ['to', 'pick', 'rags', '.'], ['belabor', 'the', 'obvious', '.'], ['bell', 'cows', '.'], ['he', 'took', 'the', 'manuscript', 'in', 'both', 'hands', 'and', 'gave', 'it', 'a', 'mighty', 'tear', '.'], ['kill', 'the', 'engine', '.'], ['analyze'

In [ ]:
model.eval()
with torch.no_grad():
  for b in it_test:
      idx1, idx2, s1, s2 = b.idx1, b.idx2, b.s1, b.s2
      pred = model(idx1, idx2, s1, s2)
      pred = torch.round(pred)


In [ ]:
with open(args.out_path, 'w') as f:
  for i in pred:
    #print(label_decode[i.item()],file=f)
    print(label_decode.get(i.item(),'F'), file=f)

In [ ]:
## DONT CHANGE THIS CELL
# Your testing code must produce a file output.txt with predictions as T and F in each line

## Final Evaluation 
# this is currently the same as dev.gold.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
correct, total = 0., 0.
for prediction, gold in zip(open('output.txt'), open('test.gold.txt')):
  prediction, gold = prediction.strip(), gold.strip()
  total += 1
  if prediction == gold:
    correct += 1

## Report this as the final validation performance 
print('Performance = ', (correct/total))

--2021-10-11 18:19:40--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt [following]
--2021-10-11 18:19:41--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1276 (1.2K) [text/plain]
Saving to: ‘test.gold.txt’

test.gold.txt       100%[===================>]   1.25K  --.-KB/s    in 0s      

2021-10-11 18:19:42 (51.1 MB/s) - ‘test.gold.txt’ saved [1276/1276]

Performance =  0.5893416927899686
